# Tutorial: Processing Marker Data

**Author:** Robbin Romijnders  
**Date Created:** Sep 28, 2021  
**Last Modified:** Sep 28, 2021

## Introduction
In this tutorial, the data files with optical motion capture data are firstly loaded, and then the marker data are subject to the following processing steps:

1. Correct for any gaps in the marker position data, 
2. Low-pass filter the marker position data,
3. Detect any instances of intial foot contact (IC) and final foot contact (FC)

In [5]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from lib.utils import _load_file
from lib.preprocessing import _predict_missing_markers, _butter_lowpass
from lib.analysis import _get_gait_events_from_OMC
import os
from scipy.signal import find_peaks

In [6]:
# Set data directory
PARENT_FOLDER = "/mnt/neurogeriatrics_data/Keep Control/Data/lab dataset"

# Get a list of participant ids
participant_ids = [folder_name for folder_name in os.listdir(PARENT_FOLDER) if folder_name.startswith("pp")]
print(participant_ids[:10])



['pp001', 'pp002', 'pp003', 'pp004', 'pp005', 'pp006', 'pp007', 'pp008', 'pp009', 'pp011']


In [8]:
# Trial to investigate
trial_name = "walk_preferred"

# Loop over the participants
for (ix_participant, participant_id) in enumerate(participant_ids[:1]):

    # Get a list of OMC data files
    omc_files = [file_name for file_name in os.listdir(os.path.join(PARENT_FOLDER, participant_id, "optical")) if file_name.endswith(".mat")]

    # Loop over the files
    for (ix_file, omc_file_name) in enumerate(omc_files):

        #
        if trial_name in omc_file_name:
            
            # Check if their is a corresponding IMU data file
            if os.path.isfile(os.path.join(PARENT_FOLDER, participant_id, "imu", omc_file_name.replace("omc_", "imu_"))):
                print(os.path.join(PARENT_FOLDER, participant_id, "optical", omc_file_name))
                print(os.path.join(PARENT_FOLDER, participant_id, "imu", omc_file_name.replace("omc_", "imu_")))
                
                # Load optical motion capture data
                omc_data = _load_file(os.path.join(PARENT_FOLDER, participant_id, "optical", omc_file_name))

                # Load inertial motion capture data
                imu_data = _load_file(os.path.join(PARENT_FOLDER, participant_id, "imu", omc_file_name.replace("omc_", "imu_")))
                break
print(f"OMC: {omc_file_name}\nIMU: {omc_file_name.replace('omc_', 'imu_')}")

/mnt/neurogeriatrics_data/Keep Control/Data/lab dataset/pp001/optical/omc_walk_preferred.mat
/mnt/neurogeriatrics_data/Keep Control/Data/lab dataset/pp001/imu/imu_walk_preferred.mat
OMC: omc_walk_preferred.mat
IMU: imu_walk_preferred.mat


We now have a dictionary, `omc_data`, that contains all the relevant information concerning our trial:
- `fs`: sampling frequency, in Hz
- `pos`: (N, 4, M) array, the marker position data, in mm; with N time steps (or frames), 3 dimensions + 1 error, M markers
- `calibration_file`: the corresponding file of the calibration procedure, in case joint angles are to be calibrated
- `marker_location`: array, with the marker positions

## Preprocessing
The marker position data are first interpolated to get rid of any missing data. Next, the data are low-pass filtered to reduce the effects of high-frequency noise.

In [9]:
# Reorganize data to comply with _predict_missing_markers() function
n_time_steps, n_markers = omc_data["pos"].shape[0], omc_data["pos"].shape[-1]
tmp = np.reshape(omc_data["pos"][:,:3,:], (n_time_steps, 3*n_markers), order="F")
tmp = _predict_missing_markers(tmp)

filled_data = np.reshape(tmp, (n_time_steps, 3, n_markers), order="F")
del tmp

In [10]:
# Low-pass filter the data
fs = omc_data["fs"]
tmp = np.reshape(filled_data, (n_time_steps, 3*n_markers), order="F")
filtered_data = _butter_lowpass(tmp, fs)
filtered_data = np.reshape(filtered_data, (n_time_steps, 3, n_markers), order="F")

In [11]:
l_mid_foot_vel, r_mid_foot_vel, l_ix_IC, l_ix_FC, r_ix_IC, r_ix_FC = _get_gait_events_from_OMC(filtered_data, fs, omc_data["marker_location"])

In [13]:
l_heel_pos = filtered_data[:,:,np.argwhere(omc_data["marker_location"]=='l_heel')[:,0][0]]
l_toe_pos = filtered_data[:,:,np.argwhere(omc_data["marker_location"]=='l_toe')[:,0][0]]
r_heel_pos = filtered_data[:,:,np.argwhere(omc_data["marker_location"]=='r_heel')[:,0][0]]
r_toe_pos = filtered_data[:,:,np.argwhere(omc_data["marker_location"]=='r_toe')[:,0][0]]

fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

# Position data
fig.add_trace(go.Scatter(x=np.arange(l_heel_pos.shape[0]), y=l_heel_pos[:,2], mode='lines', line=dict(color='rgba(0, 0, 255, 1)', width=1), name='l_heel'), row=1, col=1)
fig.add_trace(go.Scatter(x=l_ix_IC, y=l_heel_pos[l_ix_IC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(0, 0, 255, 1)')), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(l_toe_pos.shape[0]), y=l_toe_pos[:,2], mode='lines', line=dict(color='rgba(0, 0, 255, 0.3)', width=1), name='l_toe'), row=1, col=1)
fig.add_trace(go.Scatter(x=l_ix_FC, y=l_toe_pos[l_ix_FC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(0, 0, 255, 1)')), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(r_heel_pos.shape[0]), y=r_heel_pos[:,2], mode='lines', line=dict(color='rgba(255, 0, 0, 1)', width=1), name='r_heel'), row=1, col=1)
fig.add_trace(go.Scatter(x=r_ix_IC, y=r_heel_pos[r_ix_IC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(255, 0, 0, 1)')), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(r_toe_pos.shape[0]), y=r_toe_pos[:,2], mode='lines', line=dict(color='rgba(255, 0, 0, 0.3)', width=1), name='r_toe'), row=1, col=1)
fig.add_trace(go.Scatter(x=r_ix_FC, y=r_toe_pos[r_ix_FC,-1], mode="markers", marker=dict(symbol='triangle-up-open', size=8, color='rgba(255, 0, 0, 1)')), row=1, col=1)

# Velocity
fig.add_trace(go.Scatter(x=np.arange(l_mid_foot_vel.shape[0]), y=l_mid_foot_vel[:,-1], mode='lines', line=dict(color='rgba(0, 0, 255, 1)', width=1), name='l_mid_foot_vel_z'), row=2, col=1)
fig.add_trace(go.Scatter(x=l_ix_IC, y=l_mid_foot_vel[l_ix_IC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(0, 0, 255, 1)')), row=2, col=1)
fig.add_trace(go.Scatter(x=l_ix_FC, y=l_mid_foot_vel[l_ix_FC,-1], mode="markers", marker=dict(symbol='triangle-up-open', size=8, color='rgba(0, 0, 255, 1)')), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(r_mid_foot_vel.shape[0]), y=r_mid_foot_vel[:,-1], mode='lines', line=dict(color='rgba(255, 0, 0, 1)', width=1), name='r_mid_foot_vel_z'), row=2, col=1)
fig.add_trace(go.Scatter(x=r_ix_IC, y=r_mid_foot_vel[r_ix_IC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(255, 0, 0, 1)')), row=2, col=1)
fig.add_trace(go.Scatter(x=r_ix_FC, y=r_mid_foot_vel[r_ix_FC,-1], mode="markers", marker=dict(symbol='triangle-up-open', size=8, color='rgba(255, 0, 0, 1)')), row=2, col=1)

# Horizontal foot velocity
fig.add_trace(go.Scatter(x=np.arange(l_mid_foot_vel.shape[0]), y=l_mid_foot_vel[:,0], mode='lines', line=dict(width=1, color='rgba(0, 0, 255, 1)'), name='l_mid_foot_vel_x'), row=3, col=1)
fig.add_trace(go.Scatter(x=l_ix_IC, y=l_mid_foot_vel[l_ix_IC,0], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(0, 0, 255, 1)')), row=3, col=1)
fig.add_trace(go.Scatter(x=l_ix_FC, y=l_mid_foot_vel[l_ix_FC,0], mode="markers", marker=dict(symbol='triangle-up-open', size=8, color='rgba(0, 0, 255, 1)')), row=3, col=1)
fig.add_trace(go.Scatter(x=np.arange(r_mid_foot_vel.shape[0]), y=r_mid_foot_vel[:,0], mode='lines', line=dict(width=1, color='rgba(255, 0, 0, 1)'), name='r_mid_foot_vel_x'), row=3, col=1)
fig.add_trace(go.Scatter(x=r_ix_IC, y=r_mid_foot_vel[r_ix_IC,0], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(255, 0, 0, 1)')), row=3, col=1)
fig.add_trace(go.Scatter(x=r_ix_FC, y=r_mid_foot_vel[r_ix_FC,0], mode="markers", marker=dict(symbol='triangle-up-open', size=8, color='rgba(255, 0, 0, 1)')), row=3, col=1)

fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.show()

In [16]:
print(imu_data["imu_location"])
l_ank_gyro = imu_data["gyro"][:,:,np.argwhere(imu_data["imu_location"]=="left_ankle")[:,0][0]]
print(l_ank_gyro)

['head' 'sternum' 'left_upper_arm' 'left_fore_arm' 'right_upper_arm'
 'right_fore_arm' 'pelvis' 'left_thigh' 'left_shank' 'left_foot'
 'right_thigh' 'right_shank' 'right_foot' 'left_ankle' 'right_ankle'
 'pocket']
[[  0.8765316    2.44735622  -0.17798795]
 [  1.13949108   1.57452285   0.6170249 ]
 [  1.49010372   2.18493581   1.57815981]
 ...
 [-45.19981384  26.3162117   25.44041061]
 [-45.46277237  26.2306385   24.04023933]
 [-41.44533539  26.3162117   21.33482361]]


In [25]:
l_heel_pos = filtered_data[:,:,np.argwhere(omc_data["marker_location"]=='l_heel')[:,0][0]]
l_toe_pos = filtered_data[:,:,np.argwhere(omc_data["marker_location"]=='l_toe')[:,0][0]]
r_heel_pos = filtered_data[:,:,np.argwhere(omc_data["marker_location"]=='r_heel')[:,0][0]]
r_toe_pos = filtered_data[:,:,np.argwhere(omc_data["marker_location"]=='r_toe')[:,0][0]]

fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

# Position data
fig.add_trace(go.Scatter(x=np.arange(l_heel_pos.shape[0]), y=l_heel_pos[:,2], mode='lines', line=dict(color='rgba(0, 0, 255, 1)', width=1), name='l_heel'), row=1, col=1)
fig.add_trace(go.Scatter(x=l_ix_IC, y=l_heel_pos[l_ix_IC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(0, 0, 255, 1)')), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(l_toe_pos.shape[0]), y=l_toe_pos[:,2], mode='lines', line=dict(color='rgba(0, 0, 255, 0.3)', width=1), name='l_toe'), row=1, col=1)
fig.add_trace(go.Scatter(x=l_ix_FC, y=l_toe_pos[l_ix_FC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(0, 0, 255, 1)')), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(r_heel_pos.shape[0]), y=r_heel_pos[:,2], mode='lines', line=dict(color='rgba(255, 0, 0, 1)', width=1), name='r_heel'), row=1, col=1)
fig.add_trace(go.Scatter(x=r_ix_IC, y=r_heel_pos[r_ix_IC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(255, 0, 0, 1)')), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(r_toe_pos.shape[0]), y=r_toe_pos[:,2], mode='lines', line=dict(color='rgba(255, 0, 0, 0.3)', width=1), name='r_toe'), row=1, col=1)
fig.add_trace(go.Scatter(x=r_ix_FC, y=r_toe_pos[r_ix_FC,-1], mode="markers", marker=dict(symbol='triangle-up-open', size=8, color='rgba(255, 0, 0, 1)')), row=1, col=1)

# Velocity
fig.add_trace(go.Scatter(x=np.arange(l_mid_foot_vel.shape[0]), y=l_mid_foot_vel[:,-1], mode='lines', line=dict(color='rgba(0, 0, 255, 1)', width=1), name='l_mid_foot_vel_z'), row=2, col=1)
fig.add_trace(go.Scatter(x=l_ix_IC, y=l_mid_foot_vel[l_ix_IC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(0, 0, 255, 1)')), row=2, col=1)
fig.add_trace(go.Scatter(x=l_ix_FC, y=l_mid_foot_vel[l_ix_FC,-1], mode="markers", marker=dict(symbol='triangle-up-open', size=8, color='rgba(0, 0, 255, 1)')), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(r_mid_foot_vel.shape[0]), y=r_mid_foot_vel[:,-1], mode='lines', line=dict(color='rgba(255, 0, 0, 1)', width=1), name='r_mid_foot_vel_z'), row=2, col=1)
fig.add_trace(go.Scatter(x=r_ix_IC, y=r_mid_foot_vel[r_ix_IC,-1], mode="markers", marker=dict(symbol='triangle-down-open', size=8, color='rgba(255, 0, 0, 1)')), row=2, col=1)
fig.add_trace(go.Scatter(x=r_ix_FC, y=r_mid_foot_vel[r_ix_FC,-1], mode="markers", marker=dict(symbol='triangle-up-open', size=8, color='rgba(255, 0, 0, 1)')), row=2, col=1)

# Horizontal foot velocity
fig.add_trace(go.Scatter(x=np.arange(l_ank_gyro.shape[0]), y=l_ank_gyro[:,0], mode="lines", line=dict(width=1, color='rgba(255, 0, 0, 1)'), name='l_ank_x'), row=3, col=1)
fig.add_trace(go.Scatter(x=np.arange(l_ank_gyro.shape[0]), y=l_ank_gyro[:,1], mode="lines", line=dict(width=1, color='rgba(0, 255, 0, 1)'), name='l_ank_y'), row=3, col=1)
fig.add_trace(go.Scatter(x=np.arange(l_ank_gyro.shape[0]), y=l_ank_gyro[:,2], mode="lines", line=dict(width=1, color='rgba(0, 0, 255, 1)'), name='l_ank_z'), row=3, col=1)
for i in l_ix_IC:
    fig.add_trace(go.Scatter(x=[i, i], y=[np.min([np.min(l_heel_pos[:,2]), np.min(r_heel_pos[:,2])]), np.max([np.max(l_heel_pos[:,2]), np.max(r_heel_pos[:,2])])], mode='lines', line=dict(width=1, color='rgba(0,0,0,1)', dash='dot')), row=1, col=1)
    fig.add_trace(go.Scatter(x=[i, i], y=[np.min([np.min(l_mid_foot_vel[:,2]), np.min(r_mid_foot_vel[:,2])]), np.max([np.max(l_mid_foot_vel[:,2]), np.max(r_mid_foot_vel[:,2])])], mode='lines', line=dict(width=1, color='rgba(0,0,0,1)', dash='dot')), row=2, col=1)
    fig.add_trace(go.Scatter(x=[i, i], y=[np.min(np.min(l_ank_gyro)), np.max(np.max(l_ank_gyro))], mode='lines', line=dict(width=1, color='rgba(0,0,0,1)', dash='dot')), row=3, col=1)

fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.show()